# Classification Complexity Study: Circle vs Ring

A nonlinear binary classification task where the SingleDendrite's nonlinear dynamics are actually **useful**.

## Task: Circle Inside Ring

```
         ████████████
       ██            ██
      █    ░░░░░░░░    █
     █   ░░░░░░░░░░░░   █
     █   ░░░░░░░░░░░░   █     ░ = Class 0 (inner circle)
     █   ░░░░░░░░░░░░   █     █ = Class 1 (outer ring)
      █    ░░░░░░░░    █
       ██            ██
         ████████████
```

This is a **nonlinear** classification problem - a single linear classifier cannot solve it!

## Why This is Good for SOEN

Unlike linear regression, this task **benefits** from the SingleDendrite's nonlinear dynamics.
The question: How many neurons do we need to solve this?

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from soen_toolkit.core import (
    ConnectionConfig,
    LayerConfig,
    SimulationConfig,
    SOENModelCore,
)

torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")

## 1. Generate Circle-in-Ring Dataset

In [ ]:
def generate_circle_ring_data(n_samples=500, inner_radius=0.3, outer_radius_min=0.5, 
                               outer_radius_max=0.8, noise=0.05):
    """
    Generate 2D classification data: circle inside a ring.
    
    Class 0: Points inside inner circle (r < inner_radius)
    Class 1: Points in outer ring (outer_radius_min < r < outer_radius_max)
    
    Returns:
        X: [n_samples, 2] - (x, y) coordinates
        y: [n_samples] - class labels (0 or 1)
    """
    n_each = n_samples // 2
    
    # Class 0: Inner circle
    theta_inner = np.random.uniform(0, 2*np.pi, n_each)
    r_inner = np.random.uniform(0, inner_radius, n_each)
    x_inner = r_inner * np.cos(theta_inner) + np.random.normal(0, noise, n_each)
    y_inner = r_inner * np.sin(theta_inner) + np.random.normal(0, noise, n_each)
    
    # Class 1: Outer ring
    theta_outer = np.random.uniform(0, 2*np.pi, n_each)
    r_outer = np.random.uniform(outer_radius_min, outer_radius_max, n_each)
    x_outer = r_outer * np.cos(theta_outer) + np.random.normal(0, noise, n_each)
    y_outer = r_outer * np.sin(theta_outer) + np.random.normal(0, noise, n_each)
    
    # Combine
    X = np.vstack([
        np.column_stack([x_inner, y_inner]),
        np.column_stack([x_outer, y_outer])
    ])
    y = np.array([0] * n_each + [1] * n_each)
    
    # Shuffle
    idx = np.random.permutation(len(y))
    X, y = X[idx], y[idx]
    
    # Scale to SOEN operating range [0, 0.3]
    X = (X + 1) / 2 * 0.25 + 0.025  # Map [-1, 1] to [0.025, 0.275]
    
    return torch.FloatTensor(X), torch.FloatTensor(y)


# Generate data
N_SAMPLES = 500
X_data, y_data = generate_circle_ring_data(N_SAMPLES)

print(f"Dataset shape: X={X_data.shape}, y={y_data.shape}")
print(f"Class distribution: {(y_data == 0).sum().item()} inner, {(y_data == 1).sum().item()} outer")
print(f"X range: [{X_data.min():.3f}, {X_data.max():.3f}]")

# Visualize
plt.figure(figsize=(8, 8))
colors = ['blue', 'red']
for c in [0, 1]:
    mask = y_data == c
    label = 'Inner circle (class 0)' if c == 0 else 'Outer ring (class 1)'
    plt.scatter(X_data[mask, 0], X_data[mask, 1], c=colors[c], 
                alpha=0.6, s=30, label=label)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Binary Classification: Circle vs Ring')
plt.legend()
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

## 2. Prepare Data for SOEN

SOEN expects temporal sequences. We'll use constant inputs over time.

In [ ]:
SEQ_LEN = 50  # Time steps for SOEN dynamics to settle

# Expand to sequence: [N, T, 2]
X_seq = X_data.unsqueeze(1).expand(-1, SEQ_LEN, -1).clone()
y_labels = y_data.unsqueeze(1)  # [N, 1]

print(f"SOEN input shape: {X_seq.shape}")
print(f"Labels shape: {y_labels.shape}")

## 3. Model Builder for Classification

In [ ]:
def build_classifier(hidden_dims, input_dim=2, dt=50.0):
    """
    Build a SOEN classifier.
    
    Architecture: 2 (input) → hidden → 1 (output)
    
    Args:
        hidden_dims: List of hidden layer sizes
        input_dim: Input dimension (2 for x,y coordinates)
    """
    sim_cfg = SimulationConfig(
        dt=dt,
        input_type="state",
        track_phi=False,
        track_power=False,
    )
    
    layers = []
    connections = []
    
    # Input layer (dim=2 for x, y)
    layers.append(LayerConfig(
        layer_id=0,
        layer_type="Input",
        params={"dim": input_dim},
    ))
    
    # Hidden layers
    prev_dim = input_dim
    for i, hidden_dim in enumerate(hidden_dims):
        layer_id = i + 1
        
        layers.append(LayerConfig(
            layer_id=layer_id,
            layer_type="SingleDendrite",
            params={
                "dim": hidden_dim,
                "solver": "FE",
                "source_func": "Heaviside_fit_state_dep",
                "phi_offset": 0.02,
                "bias_current": 1.98,
                "gamma_plus": 0.0005,
                "gamma_minus": 1e-6,
                "learnable_params": {
                    "phi_offset": False,
                    "bias_current": False,
                    "gamma_plus": False,
                    "gamma_minus": False,
                },
            },
        ))
        
        connections.append(ConnectionConfig(
            from_layer=layer_id - 1,
            to_layer=layer_id,
            connection_type="all_to_all",
            learnable=True,
            params={"init": "xavier_uniform"},
        ))
        
        prev_dim = hidden_dim
    
    # Output layer (dim=1 for binary classification)
    output_layer_id = len(hidden_dims) + 1
    layers.append(LayerConfig(
        layer_id=output_layer_id,
        layer_type="Input",
        params={"dim": 1},
    ))
    
    connections.append(ConnectionConfig(
        from_layer=output_layer_id - 1,
        to_layer=output_layer_id,
        connection_type="all_to_all",
        learnable=True,
        params={"init": "xavier_uniform"},
    ))
    
    model = SOENModelCore(
        sim_config=sim_cfg,
        layers_config=layers,
        connections_config=connections,
    )
    
    return model


def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Test builder
print("Testing classifier builder...")
test_configs = [
    ([2], "2→2→1"),
    ([4], "2→4→1"),
    ([8], "2→8→1"),
    ([4, 4], "2→4→4→1"),
]

for hidden_dims, name in test_configs:
    model = build_classifier(hidden_dims)
    n_params = count_params(model)
    layer_dims = [l.dim for l in model.layers]
    print(f"  {name}: layers={layer_dims}, params={n_params}")

## 4. Training Function for Classification

In [ ]:
def train_classifier(model, X_train, y_train, n_epochs=300, lr=0.02, verbose=False):
    """
    Train SOEN classifier with BCE loss.
    """
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    
    losses = []
    accuracies = []
    
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        # Forward
        final_hist, _ = model(X_train)
        logits = final_hist[:, -1, :]  # [N, 1]
        
        # Loss
        loss = criterion(logits, y_train)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Metrics
        with torch.no_grad():
            preds = (torch.sigmoid(logits) > 0.5).float()
            acc = (preds == y_train).float().mean().item()
        
        losses.append(loss.item())
        accuracies.append(acc)
        
        if verbose and (epoch + 1) % 50 == 0:
            print(f"  Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")
    
    return losses, accuracies


def evaluate_classifier(model, X_test, y_test):
    """
    Evaluate classifier and return predictions.
    """
    model.eval()
    with torch.no_grad():
        final_hist, _ = model(X_test)
        logits = final_hist[:, -1, :]
        probs = torch.sigmoid(logits).squeeze().numpy()
        preds = (probs > 0.5).astype(float)
    
    y_true = y_test.squeeze().numpy()
    accuracy = (preds == y_true).mean()
    
    return preds, probs, accuracy

## 5. Train All Architectures

In [ ]:
# Architectures to compare (input_dim=2 for all)
ARCHITECTURES = {
    "2 neurons": [2],
    "4 neurons": [4],
    "8 neurons": [8],
    "16 neurons": [16],
    "32 neurons": [32],
    "4→4 (deep)": [4, 4],
    "8→8 (deep)": [8, 8],
    "4→4→4 (3 layers)": [4, 4, 4],
}

N_EPOCHS = 400
LR = 0.02

results = {}

print("Training all architectures...")
print("=" * 60)

for name, hidden_dims in ARCHITECTURES.items():
    print(f"\nTraining: {name}")
    
    model = build_classifier(hidden_dims)
    n_params = count_params(model)
    
    losses, accuracies = train_classifier(
        model, X_seq, y_labels, n_epochs=N_EPOCHS, lr=LR, verbose=False
    )
    
    preds, probs, final_acc = evaluate_classifier(model, X_seq, y_labels)
    
    results[name] = {
        'hidden_dims': hidden_dims,
        'n_params': n_params,
        'losses': losses,
        'accuracies': accuracies,
        'final_acc': final_acc,
        'final_loss': losses[-1],
        'preds': preds,
        'probs': probs,
        'model': model,
    }
    
    print(f"  Params: {n_params}, Accuracy: {final_acc:.4f}")

print("\n" + "=" * 60)
print("Training complete!")

## 6. Compare Training Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0, 1, len(results)))

# Loss curves
ax1 = axes[0]
for (name, res), color in zip(results.items(), colors):
    ax1.plot(res['losses'], label=f"{name}", color=color, lw=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('BCE Loss')
ax1.set_title('Training Loss')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# Accuracy curves
ax2 = axes[1]
for (name, res), color in zip(results.items(), colors):
    ax2.plot(res['accuracies'], label=f"{name}", color=color, lw=2)
ax2.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Perfect')
ax2.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Random')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy')
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0.4, 1.05)

plt.tight_layout()
plt.show()

## 7. Decision Boundary Visualization

In [ ]:
def plot_decision_boundary(model, X_data, y_data, ax, title, resolution=100):
    """
    Plot decision boundary for a 2D classifier.
    """
    # Create grid
    x_min, x_max = X_data[:, 0].min() - 0.02, X_data[:, 0].max() + 0.02
    y_min, y_max = X_data[:, 1].min() - 0.02, X_data[:, 1].max() + 0.02
    
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, resolution),
        np.linspace(y_min, y_max, resolution)
    )
    
    # Prepare grid as SOEN input
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_tensor = torch.FloatTensor(grid_points)
    grid_seq = grid_tensor.unsqueeze(1).expand(-1, SEQ_LEN, -1).clone()
    
    # Get predictions
    model.eval()
    with torch.no_grad():
        final_hist, _ = model(grid_seq)
        logits = final_hist[:, -1, :]
        probs = torch.sigmoid(logits).squeeze().numpy()
    
    Z = probs.reshape(xx.shape)
    
    # Plot
    ax.contourf(xx, yy, Z, levels=50, cmap='RdBu', alpha=0.7)
    ax.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    
    # Plot data points
    for c, color in enumerate(['blue', 'red']):
        mask = y_data.squeeze() == c
        ax.scatter(X_data[mask, 0], X_data[mask, 1], c=color, 
                   s=20, alpha=0.6, edgecolors='white', linewidths=0.5)
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title)
    ax.set_aspect('equal')


# Plot decision boundaries for all architectures
n_models = len(results)
cols = 4
rows = (n_models + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))
axes = axes.flatten()

X_np = X_data.numpy()
y_np = y_data.numpy()

for idx, (name, res) in enumerate(results.items()):
    ax = axes[idx]
    plot_decision_boundary(
        res['model'], X_np, y_np, ax,
        f"{name}\nAcc={res['final_acc']:.3f}, Params={res['n_params']}"
    )

# Hide unused
for idx in range(n_models, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

## 8. Performance vs Complexity

In [ ]:
names = list(results.keys())
n_params = [results[n]['n_params'] for n in names]
accuracies = [results[n]['final_acc'] for n in names]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0, 1, len(results)))

# Bar chart
ax1 = axes[0]
bars = ax1.bar(range(len(names)), accuracies, color=colors)
ax1.axhline(y=0.5, color='gray', linestyle='--', label='Random (50%)')
ax1.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='Perfect (100%)')
ax1.set_xticks(range(len(names)))
ax1.set_xticklabels(names, rotation=45, ha='right')
ax1.set_ylabel('Accuracy')
ax1.set_title('Final Accuracy by Architecture')
ax1.set_ylim(0.4, 1.05)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

for bar, acc in zip(bars, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontsize=9)

# Scatter: params vs accuracy
ax2 = axes[1]
ax2.scatter(n_params, accuracies, s=150, c=colors, edgecolors='black', zorder=5)
for i, name in enumerate(names):
    ax2.annotate(name, (n_params[i], accuracies[i]), 
                 textcoords="offset points", xytext=(5, 5), fontsize=8)
ax2.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Number of Trainable Parameters')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracy vs Model Complexity')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0.4, 1.05)

plt.tight_layout()
plt.show()

## 9. Best vs Worst Model Comparison

In [ ]:
# Find best and worst
best_name = max(results, key=lambda x: results[x]['final_acc'])
worst_name = min(results, key=lambda x: results[x]['final_acc'])

print(f"Best: {best_name} (Accuracy: {results[best_name]['final_acc']:.4f})")
print(f"Worst: {worst_name} (Accuracy: {results[worst_name]['final_acc']:.4f})")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, name, title in zip(axes, [worst_name, best_name], ['Worst (Simplest)', 'Best']):
    res = results[name]
    plot_decision_boundary(
        res['model'], X_np, y_np, ax,
        f"{title}: {name}\nAccuracy={res['final_acc']:.4f}, Params={res['n_params']}"
    )

plt.tight_layout()
plt.show()

## 10. Summary Table

In [ ]:
import pandas as pd

summary_data = []
for name, res in results.items():
    n_hidden = sum(res['hidden_dims'])
    n_layers = len(res['hidden_dims'])
    
    summary_data.append({
        'Architecture': name,
        'Hidden Neurons': n_hidden,
        'Hidden Layers': n_layers,
        'Trainable Params': res['n_params'],
        'Final Accuracy': f"{res['final_acc']:.4f}",
        'Final Loss': f"{res['final_loss']:.4f}",
    })

df = pd.DataFrame(summary_data)
df = df.sort_values('Trainable Params')

print("=" * 90)
print("CLASSIFICATION COMPLEXITY STUDY: SUMMARY")
print("=" * 90)
print(f"\nTask: Binary classification (Circle vs Ring)")
print(f"Input dimension: 2 (x, y coordinates)")
print(f"Training samples: {N_SAMPLES}")
print(f"Training epochs: {N_EPOCHS}")
print()
print(df.to_string(index=False))
print("=" * 90)

## 11. Conclusions

In [ ]:
print("=" * 70)
print("CONCLUSIONS")
print("=" * 70)

# Analyze trends
wide_models = ['2 neurons', '4 neurons', '8 neurons', '16 neurons', '32 neurons']
wide_accs = [results[n]['final_acc'] for n in wide_models if n in results]

print("\n1. EFFECT OF WIDTH (more neurons in hidden layer):")
for name in wide_models:
    if name in results:
        print(f"   {name}: Accuracy = {results[name]['final_acc']:.4f}")

if wide_accs[-1] > wide_accs[0]:
    improvement = (wide_accs[-1] - wide_accs[0]) * 100
    print(f"   → More neurons HELPS (+{improvement:.1f}% accuracy)")

print("\n2. EFFECT OF DEPTH:")
if '4→4 (deep)' in results and '8 neurons' in results:
    shallow_8 = results['8 neurons']['final_acc']
    deep_4x4 = results['4→4 (deep)']['final_acc']
    print(f"   8 neurons (1 layer):  Acc = {shallow_8:.4f}, Params = {results['8 neurons']['n_params']}")
    print(f"   4→4 (2 layers):       Acc = {deep_4x4:.4f}, Params = {results['4→4 (deep)']['n_params']}")
    if deep_4x4 > shallow_8:
        print(f"   → Depth helps with similar parameter count!")

print("\n3. COMPARISON TO LINEAR REGRESSION:")
print("   Unlike linear regression, this nonlinear classification task")
print("   BENEFITS from the SingleDendrite's nonlinear dynamics.")
print(f"   Best accuracy achieved: {results[best_name]['final_acc']:.4f}")

print("\n4. HARDWARE REQUIREMENTS FOR BEST MODEL:")
best_res = results[best_name]
print(f"   Architecture: {best_name}")
print(f"   SingleDendrite neurons: {sum(best_res['hidden_dims'])}")
print(f"   Trainable connections: {best_res['n_params']}")

print("\n5. KEY INSIGHT:")
if results[best_name]['final_acc'] > 0.9:
    print("   ✓ SOEN can effectively solve nonlinear classification!")
    print("   ✓ The nonlinear dynamics are an ASSET for this task.")
else:
    print("   More neurons or training may be needed for higher accuracy.")

print("\n" + "=" * 70)